# Press Spacebar to start simulation

In [1]:
#WORKING improved sense follow check
import numpy as np
import pygame
import sys
from scipy.ndimage import gaussian_filter
import cv2

pygame.init()

width, height = 1536//4, 814//4
screen = pygame.display.set_mode((width, height))
pygame.display.set_caption("Slime")

clock = pygame.time.Clock()
running = True

np.random.seed(123)

particlenumber=2000

anglerandomnessfactor=5

vel=1

pixels = np.zeros((width, height, 3), dtype=np.uint8)
pixelsfollow = np.zeros((width, height, 3), dtype=np.uint8)
posfollow =  np.ones((particlenumber,2))
isfollowing = np.zeros(particlenumber)
angle = np.random.uniform(0, 2*np.pi, size=(particlenumber))
pos = np.ones((particlenumber,2))
pos1 = np.ones((particlenumber,2))
# pos[:,0] =  np.random.uniform(1, width-1, size=(particlenumber))
# pos[:,1] = np.random.uniform(1, height-1, size=(particlenumber))

# pos[:,0] =  width//2
# pos[:,1] = height//2

def is_inside_circle(x, y, circle_radius):
    return (x - width / 2) ** 2 + (y - height / 2) ** 2 <= circle_radius ** 2

# Define the radius of the circle
circle_radius = min(width, height) / 2.5

# Generate random polar coordinates within the circle
r = np.random.uniform(0, circle_radius, size=(particlenumber))
theta = np.random.uniform(0, 2 * np.pi, size=(particlenumber))

# Convert polar coordinates to Cartesian coordinates
pos[:, 0] = width / 2 + r * np.cos(theta)
pos[:, 1] = height / 2 + r * np.sin(theta)

for i in range(particlenumber):
    pos1[i, 0] = pos[i, 0] + np.cos(angle[i]) * vel*10
    pos1[i, 1] = pos[i, 1] + np.sin(angle[i]) * vel*10

    # Check if the particle has escaped the circle
    if not is_inside_circle(pos1[i, 0], pos1[i, 1], circle_radius):
        # Reset position inside the circle
        r = np.random.uniform(0, circle_radius)
        theta = np.random.uniform(0, 2 * np.pi)
        pos[i, 0] = width / 2 + r * np.cos(theta)
        pos[i, 1] = height / 2 + r * np.sin(theta)

        # Update angle towards the center
        angle[i] = np.arctan2(height / 2 - pos[i, 1], width / 2 - pos[i, 0])

pos[:,0] =  np.random.uniform(1, width-1, size=(particlenumber))
pos[:,1] = np.random.uniform(1, height-1, size=(particlenumber))




a=0
        
def nupd():
    for i in range(particlenumber):
        pos[i,0]=pos[i,0]+np.cos(angle[i])*vel
        pos[i,1]=pos[i,1]+np.sin(angle[i])*vel
        if (pos[i,1]>=height or pos[i,1]<0 or pos[i,0]>=width or pos[i,0]<0):
            pos[i,0]=pos[i,0]-np.cos(angle[i])*vel
            pos[i,1]=pos[i,1]-np.sin(angle[i])*vel
            angle[i] = np.random.uniform(0, 2*np.pi)
        
        
        cc=0
        maxx=0
        maxang=0
        
        res=3
        pixel_col=np.zeros((res,4))
        reach=np.pi/2
        
        
        for degr in range(res):
            deg=(reach/(res-1)*degr)-reach/2
#             print(degr)
#             print(deg/2/np.pi*360)
            xtpos=int(pos[i,0]+np.cos(angle[i]+deg)*(vel*3))
            ytpos=int(pos[i,1]+np.sin(angle[i]+deg)*(vel*3))
            if (ytpos<height and ytpos>=0 and xtpos<width and xtpos>=0):
                pixel_col[cc] = screen.get_at((int(pos[i,0]+np.cos(angle[i]+deg)*(vel*3)), int(pos[i,1]+np.sin(angle[i]+deg)*(vel*3))))
            else:
                pixel_col[cc] = np.zeros(4)
            
            l=np.mean(pixel_col[cc,:3])
            if (l>maxx):
                maxx=l
                maxang=deg
            
            cc+=1
        #l=np.mean((pixel_col[:,:3]),axis=1)
        
        if (maxx>0):
            #angle[i]=angle[i]+maxang+deg*np.random.uniform(-anglerandomnessfactor/180*np.pi, anglerandomnessfactor/180*np.pi)*(1-maxx/255)
            angle[i]=angle[i]+maxang+np.random.normal(0, anglerandomnessfactor/180*np.pi)
            isfollowing[i]=1
        else:
            isfollowing[i]=0


            
rrr=1
st=0
framno=600
fram=0
frame_ar=np.zeros((framno, width, height, 3), dtype=np.uint8)


while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.KEYDOWN and event.key == pygame.K_SPACE:
            st=1
        elif event.type == pygame.MOUSEBUTTONDOWN:
            rrr=1-rrr
            if(rrr==0):
                mp = pygame.mouse.get_pos()
                moddist=np.sqrt((mp[0]-pos[:,0])**2+(mp[1]-pos[:,1])**2)
                closest=np.argmin(moddist)
                pixelsfollow=pixels
                xx1=int((1-border)/2*width)+(pos[closest,0]*border).astype(int)
                yy1=int((1-border)/2*height)+(pos[closest,1]*border).astype(int)
                pixelsfollow[xx1-1:xx1+1,yy1-1:yy1+1] = (255,255,0)

                num_points = int(np.min(moddist))
                linex = np.linspace(pos[closest, 0], mp[0], num_points)
                liney = np.linspace(pos[closest, 1], mp[1], num_points)

                pixelsfollow[linex.astype(int), liney.astype(int)] = (255, 255, 0)
                
                print(f"Particle Info: P.No.: {closest}   Following: {isfollowing[closest]==1}   Coords: {pos[closest]}")
                
            if(rrr==1):
                pixelsfollow[linex.astype(int), liney.astype(int)] = (0, 0, 0)

    
    
    border=1
    thres=20
    
    if(rrr==1 and st==1):
    
        pixels=(pixels*0.99).astype(int)
        pixels=(pixels>thres)*pixels


        pixels = gaussian_filter(pixels, sigma=0.5)


        pixels[int((1-border)/2*width)+(pos[:,0]*border).astype(int),int((1-border)/2*height)+(pos[:,1]*border).astype(int)] = 255


        nupd()
    
        pygame.surfarray.blit_array(screen, pixels)
        

#     mp = pygame.mouse.get_pos()
#     pixels[mp[0]-10:mp[0]+10,mp[1]-10:mp[1]+10]=255
    
    if(rrr==0):
        pygame.surfarray.blit_array(screen, pixelsfollow)

    pygame.display.flip()
    clock.tick(60)

pygame.quit()
sys.exit()


pygame 2.5.2 (SDL 2.28.3, Python 3.9.13)
Hello from the pygame community. https://www.pygame.org/contribute.html


SystemExit: 

C:\Users\pranj\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3465: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
